<h1>Projeto 1 - Ciência dos Dados</h1>

Nome: *Caio Emmanuel*

Nome: *Larissa*

<h3>Introdução</h3>

Para realizar este trabalho, a dupla escolhe como tema a ser pesquisado as Olimpíadas de 2020 que ocorrerão no Japão por este ser um tema atual devido à preparação em velocidade recordista que o país teve para receber o evento e agora mais ainda devido à doença viral Corona que ameaça a realização do mesmo.

Classificamos os *tweets* obtidos como 0 ou 1, sendo 0 *tweets* irrelevantes sobre o assunto e 1 o contrário.

Vale ressaltar que usamos o artifício matemático probabilístico conhecido como *método de Naive-Bayes* para fazer previsões mais precisas sobre uma base de dados de teste.

<h3>Critérios de classificação</h3>

Classificamos como relevante aqueles *tweets* que remetem a assuntos atuais sobre a Olimpíada, como os atletas convidados, possível cancelamento, etc.

E classificamos como irrelevante aqueles que não têm relação com o evento mas tem com algum homônimo (olimpíadas científicas, regionais, olimpíadas passadas,etc.), além daqueles que julgamos não acrescentar nada ao assunto. 

<h3>Carregando algumas bibliotecas e o dataset:</h3>

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/home/caio/Documents/cdados/cdados_proj1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
dados_treino = pd.read_excel('olympics.xlsx')

In [4]:
dados_treino.head()

,Treinamento,Classification
0,rt @saitomri: i wrote about baseball. really. ...,0
1,rt @ndidi_amakaa: @solaadio olu leave your mot...,0
2,@wintersdancer they don’t want to lose the oly...,1
3,exclusive: indonesia considers 2032 olympics b...,0
4,the tentacles of canceling the tokyo olympics ...,1


<h3>Realizando limpeza dos dados</h3>

In [5]:
import re 

# Função que remove pontuação
def cleanup(text):
    punctuation = '[\/!-.:?;]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

for i in range(500):
    # Removendo pontuação
    dados_treino = dados_treino.replace(dados_treino['Treinamento'][i], cleanup(dados_treino['Treinamento'][i]))
    
    # Removendo '\n'
    dados_treino['Treinamento'] = dados_treino['Treinamento'].str.replace('[\n]','')
    
    # Removendo 'rt'
    if dados_treino['Treinamento'][i][:2] == 'rt':
        dados_treino = dados_treino.replace(dados_treino['Treinamento'][i], dados_treino['Treinamento'][i][3:])
        
    # Deixando todas as letras minúsculas:
    dados_treino = dados_treino.replace(dados_treino['Treinamento'][i], dados_treino['Treinamento'][i].lower())

In [6]:
dados_treino

,Treinamento,Classification
0,@saitomri i wrote about baseball really httpst...,0
1,@ndidi_amakaa @solaadio olu leave your mother ...,0
2,@wintersdancer they don’t want to lose the oly...,1
3,exclusive indonesia considers 2032 olympics bi...,0
4,the tentacles of canceling the tokyo olympics ...,1
...,...,...
495,highspeed camera used by nazis to both accurat...,0
496,indonesia considers 2032 olympics bid for new ...,0
497,@gailwalden6 i would like to apply to be in th...,0
498,the year is 2024 🔮womens world no 1 is 👩______...,0


<h3>Separando o dataset em *tweets* relavantes e irrelevantes</h3>

In [7]:
# tweets relevantes
tweets_r = dados_treino.loc[(dados_treino.Classification == 1)]
tweets_r

,Treinamento,Classification
2,@wintersdancer they don’t want to lose the oly...,1
4,the tentacles of canceling the tokyo olympics ...,1
5,it is the olympic year and we as a team have p...,1
9,@imrahultrehan from practicing muay thai in he...,1
10,athletes aiming for gold medals or sports busi...,1
...,...,...
478,@skift olympics cancellation would spell heavy...,1
480,olympics torch lighting ceremony will be close...,1
485,a lot of people stand to lose if olympics are ...,1
487,tokyo 2020 has been dealt its first significan...,1


In [8]:
# tweets irrelevantes
tweets_ir = dados_treino.loc[(dados_treino.Classification == 0)]

Logo a probabilidade de um *tweet* ser relevante (P_r) e irrelevante (P_ir) são...

In [12]:
P_r = tweets_r.Treinamento.count()/dados_treino.Treinamento.count()
P_ir = tweets_ir.Treinamento.count()/dados_treino.Treinamento.count()
print('A frequência de tweets relevantes é: {0}' .format(P_r))
print('A frequência de tweets irrelevantes é: {0}' .format(P_ir))

A frequência de tweets relevantes é: 0.356
A frequência de tweets irrelevantes é: 0.644


<h3>Calculando frequências relativas dentro de cada grupo</h3>

In [14]:
# criando texto com todos os tweets relevantes
txt_r = ''
for t in tweets_r['Treinamento']:
    t = str(t)
    txt_r += t

# criando série com as palavras dos tweets relevantes
words_r = pd.Series(txt_r.split())
words_r

0        @wintersdancer
1                  they
2                 don’t
3                  want
4                    to
             ...       
3634            torture
3635                yes
3636                for
3637                him
3638    @kagutamuseveni
Length: 3639, dtype: object

In [15]:
words_r = words_r.value_counts(True)
words_r

the          0.061830
olympics     0.035999
to           0.028030
of           0.019236
for          0.018137
               ...   
between      0.000275
moderates    0.000275
beyond       0.000275
@skift       0.000275
june         0.000275
Length: 1378, dtype: float64

In [16]:
words_r.sum()

1.0

In [17]:
# Do mesmo jeito para os tweets irrelevantes:
txt_ir = ''
for t in tweets_ir['Treinamento']:
    t = str(t)
    txt_ir += t
    
words_ir = pd.Series(txt_ir.split())
words_ir = words_ir.value_counts(True)
words_ir

the                        0.050114
olympics                   0.028047
to                         0.024060
and                        0.017796
in                         0.017227
                             ...   
immediately                0.000142
shoreline                  0.000142
olympics@realjameswoods    0.000142
nose                       0.000142
egypt                      0.000142
Length: 2672, dtype: float64

Assuma agora que as palavras que aparecem nos dois datasets anteriores sejam todas as palavras do Universo no Twitter (tweeverse)

In [18]:
txt = txt_r + txt_ir
tweeverse_words = pd.Series(txt.split())
tweeverse_words_frequencie = tweeverse_words.value_counts(True)

In [19]:
tweeverse_words_frequencie

the              0.054112
olympics         0.030761
to               0.025415
for              0.017256
of               0.017068
                   ...   
therethink       0.000094
system           0.000094
kelly            0.000094
energy           0.000094
automatically    0.000094
Length: 3514, dtype: float64

<h3>Realizando testes</h3>

In [20]:
dados_teste = pd.read_excel('teste.xlsx')
dados_teste

,Teste,Classification
0,fwit olympics https://t.co/72go6l7pkx,0
1,@jimodonnell2 it's certainly a supportable opi...,1
2,@kapil857 @kapil857 any link to know how point...,0
3,rt @media_sai: what a superb performance by @s...,0
4,rt @theeconomist: japan may have to cancel the...,1
...,...,...
95,study: tokyo 2020 cancellation could see japan...,1
96,japanese doctors report sharp falls in cases o...,0
97,"rt @jimmfelton: “virus is bad, might have to c...",0
98,rt @drjitendrasingh: eight indian boxers inclu...,1


Vamos realizar todo o processo de formatação e encontra P_r e P_ir dos dados de treino

In [22]:
for i in range(100):
    # Removendo pontuação
    dados_teste = dados_teste.replace(dados_teste['Teste'][i], cleanup(dados_teste['Teste'][i]))
    
    # Removendo '\n'
    dados_teste['Teste'] = dados_teste['Teste'].str.replace('[\n]','')
    
    # Removendo 'rt'
    if dados_teste['Teste'][i][:2] == 'rt':
        dados_teste = dados_teste.replace(dados_teste['Teste'][i], dados_teste['Teste'][i][3:])
        
    # Deixando todas as letras minúsculas:
    dados_teste = dados_teste.replace(dados_teste['Teste'][i], dados_teste['Teste'][i].lower())

In [23]:
dados_teste

,Teste,Classification
0,fwit olympics httpstco72go6l7pkx,0
1,@jimodonnell2 its certainly a supportable opin...,1
2,@kapil857 @kapil857 any link to know how point...,0
3,@media_sai what a superb performance by @simra...,0
4,@theeconomist japan may have to cancel the oly...,1
...,...,...
95,study tokyo 2020 cancellation could see japan’...,1
96,japanese doctors report sharp falls in cases o...,0
97,@jimmfelton “virus is bad might have to cancel...,0
98,@drjitendrasingh eight indian boxers including...,1


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**